# TODO
1. change get_baseline_model and get_xgb_model() so that instead we have a function that takes in an estimator and returns a whole pipeline
2. rewrite the SensorFusionClassifier to take an instance of the base model instead of a class

In [46]:
import pandas as pd
import numpy as np
import time
import importlib.machinery
import sys
sys.path.append('/home/sac086/extrasensory/')
import extrasense as es
from sklearn.metrics import accuracy_score, make_scorer, roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Imputer
from sklearn.model_selection import GroupShuffleSplit, GroupKFold, cross_val_score, \
                                    GridSearchCV, RandomizedSearchCV, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin, clone

In [2]:
import xgboost as xgb

# Load data

In [3]:
features_df = es.get_impersonal_data(leave_users_out=[], drop_nan_rows=False, sensors=None, label_type="activity", labeled_only=True)

timestamps = features_df.pop('timestamp')
label_source = features_df.pop("label_source")
labels = features_df.pop("label")
user_ids = features_df.pop("user_id")

In [4]:
for l in labels.unique():
    print(l)

SITTING
FIX_walking
LYING_DOWN
FIX_running
BICYCLING
STAIRS


In [5]:
folds = es.get_uids_from_es_folds()

In [22]:
def make_pipeline(clf, **params):
    steps = []
    steps.append(('imputer', Imputer(missing_values='NaN', strategy='mean', axis=0)))
    steps.append(('standardize', StandardScaler()))
    steps.append(('clf', clf(**params)))
    model = Pipeline(steps)
    return model

In [23]:
clf = make_pipeline(LogisticRegression, **dict(class_weight='balanced'))

In [8]:
def get_train_test_ind(test_fold_uids, all_user_ids):
    bool_arr = all_user_ids.isin(test_fold_uids)
    test_ind = all_user_ids.index[bool_arr]
    bool_arr = np.logical_not(bool_arr)
    train_ind = all_user_ids.index[bool_arr]
    return train_ind, test_ind
    

In [9]:
def get_metrics(y, y_pred, verbose=True):
    predictions = []
    # Naive accuracy (correct classification rate):
    accuracy = np.mean(y_pred == y);
    
    # Count occorrences of true-positive, true-negative, false-positive, and false-negative:
    tp = np.sum(np.logical_and(y_pred,y));
    tn = np.sum(np.logical_and(np.logical_not(y_pred),np.logical_not(y)));
    fp = np.sum(np.logical_and(y_pred,np.logical_not(y)));
    fn = np.sum(np.logical_and(np.logical_not(y_pred),y));
    
    # Sensitivity (=recall=true positive rate) and Specificity (=true negative rate):
    sensitivity = float(tp) / (tp+fn);
    specificity = float(tn) / (tn+fp);
    
    # Balanced accuracy is a more fair replacement for the naive accuracy:
    balanced_accuracy = (sensitivity + specificity) / 2.;
    
    # Precision:
    # Beware from this metric, since it may be too sensitive to rare labels.
    # In the ExtraSensory Dataset, there is large skew among the positive and negative classes,
    # and for each label the pos/neg ratio is different.
    # This can cause undesirable and misleading results when averaging precision across different labels.
    precision = float(tp) / (tp+fp);
    
    if verbose:
        print("-"*10);
        print('Accuracy*:         %.2f' % accuracy);
        print('Sensitivity (TPR): %.2f' % sensitivity);
        print('Specificity (TNR): %.2f' % specificity);
        print('Balanced accuracy: %.2f' % balanced_accuracy);
        print('Precision**:       %.2f' % precision);
        print("-"*10);
        
    return {'sensitivity' : sensitivity,
            'specificity' : specificity,
            'accuracy' : accuracy,
            'balanced accuracy' : balanced_accuracy,
            'precision' : precision}

In [10]:
def test_model(model_getter, context, **params):
    folds = es.get_uids_from_es_folds()
    fold_metrics = []
    
    for i, kf in enumerate(folds):
        print('Fold #%s' % i)
        model = model_getter(**params)
        
        train_ind, test_ind = get_train_test_ind(kf, user_ids)
        
        print("Training model...")
        X_train = features_df.iloc[train_ind]
        y_train = labels.iloc[train_ind]
        y_train = np.array([1 if y == context else 0 for y in y_train])
        
        X_test = features_df.iloc[test_ind]
        y_test = labels.iloc[test_ind]
        y_test = np.array([1 if y == context else 0 for y in y_test])

        model.fit(X_train, y_train)
        
        print("Testing model...")
        y_pred = model.predict(X_test)
        
        metrics = get_metrics(y_test, y_pred, verbose=True)
        fold_metrics.append(metrics)
    
    return fold_metrics

In [20]:
test_metrics = test_model(get_baseline_model, 'SITTING')

Fold #0
Training model...
Testing model...
----------
Accuracy*:         0.78
Sensitivity (TPR): 0.82
Specificity (TNR): 0.74
Balanced accuracy: 0.78
Precision**:       0.73
----------
Fold #1
Training model...
Testing model...
----------
Accuracy*:         0.74
Sensitivity (TPR): 0.83
Specificity (TNR): 0.67
Balanced accuracy: 0.75
Precision**:       0.67
----------
Fold #2
Training model...
Testing model...
----------
Accuracy*:         0.78
Sensitivity (TPR): 0.77
Specificity (TNR): 0.78
Balanced accuracy: 0.78
Precision**:       0.73
----------
Fold #3
Training model...
Testing model...
----------
Accuracy*:         0.77
Sensitivity (TPR): 0.72
Specificity (TNR): 0.79
Balanced accuracy: 0.76
Precision**:       0.69
----------
Fold #4
Training model...
Testing model...
----------
Accuracy*:         0.67
Sensitivity (TPR): 0.68
Specificity (TNR): 0.67
Balanced accuracy: 0.67
Precision**:       0.68
----------


In [11]:
def get_mean_metrics(metrics, verbose=True):
    mean_metrics = {}
    
    for fold_metrics in metrics:
        for key, val in fold_metrics.items():
            if key in mean_metrics:
                mean_metrics[key].append(val)
            else:
                mean_metrics[key] = [val]
    
    print("-"*10);
    print('Accuracy*:         %.2f' % np.mean(mean_metrics['accuracy']));
    print('Sensitivity (TPR): %.2f' % np.mean(mean_metrics['sensitivity']));
    print('Specificity (TNR): %.2f' % np.mean(mean_metrics['specificity']))
    print('Balanced accuracy: %.2f' % np.mean(mean_metrics['balanced accuracy']))
    print('Precision**:       %.2f' % np.mean(mean_metrics['precision']))
    print("-"*10);

In [23]:
get_mean_metrics(test_metrics)

----------
Accuracy*:         0.75
Sensitivity (TPR): 0.77
Specificity (TNR): 0.73
Balanced accuracy: 0.75
Precision**:       0.70
----------


In [24]:
from collections import Counter
c = Counter(labels)
for key, val in c.items():
    print("%s : %s" % (key, val))

BICYCLING : 5020
SITTING : 136728
LYING_DOWN : 141461
STAIRS : 822
FIX_walking : 22136
FIX_running : 1090


In [12]:
def get_xgb_model():
    steps = []
    steps.append(('standardize', StandardScaler()))
    steps.append(('clf', xgb.XGBClassifier()))
    model = Pipeline(steps)
    return model

In [18]:
test_metrics = test_model(get_xgb_model, 'SITTING')

Fold #0
Training model...
Testing model...
----------
Accuracy*:         0.67
Sensitivity (TPR): 0.62
Specificity (TNR): 0.72
Balanced accuracy: 0.67
Precision**:       0.66
----------
Fold #1
Training model...
Testing model...
----------
Accuracy*:         0.66
Sensitivity (TPR): 0.59
Specificity (TNR): 0.71
Balanced accuracy: 0.65
Precision**:       0.63
----------
Fold #2
Training model...
Testing model...
----------
Accuracy*:         0.70
Sensitivity (TPR): 0.64
Specificity (TNR): 0.75
Balanced accuracy: 0.69
Precision**:       0.66
----------
Fold #3
Training model...
Testing model...
----------
Accuracy*:         0.69
Sensitivity (TPR): 0.67
Specificity (TNR): 0.70
Balanced accuracy: 0.69
Precision**:       0.59
----------
Fold #4
Training model...
Testing model...
----------
Accuracy*:         0.59
Sensitivity (TPR): 0.61
Specificity (TNR): 0.58
Balanced accuracy: 0.59
Precision**:       0.60
----------


In [19]:
get_mean_metrics(test_metrics)

----------
Accuracy*:         0.66
Sensitivity (TPR): 0.62
Specificity (TNR): 0.69
Balanced accuracy: 0.66
Precision**:       0.63
----------


# Sensor Fusion
* This is from the [Recognizing Detailed Human Context-In-the-Wild from Smartphones and Smartwatches](http://extrasensory.ucsd.edu/papers/vaizman2017a_pervasiveAcceptedVersion.pdf)

In [13]:
import inspect

### [used this sklearn github issue as a helper](https://github.com/scikit-learn/scikit-learn/issues/6079#issuecomment-166990309)

In [63]:
class SensorFusionClassifier(BaseEstimator, ClassifierMixin):
    """Late Fusion Using Average Probability
    
    Attributes: 
    clf: The base classifier to use for the individual classifiers
    """
    
    base_clf = None
    base_params=None
    meta_clf = None
    meta_clf_params = None
    confidence_threshold = None
    sensors = None
    verbose=True
    ensemble_type = None
    classifiers = {}
    training_accuracy = None

    
    def __init__(self, base_clf=None, base_params=None, meta_clf=None, \
                   meta_clf_params=None, confidence_threshold=0.5, \
                   sensors=None, verbose=True, ensemble_type='bagging'):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        for arg, val in values.items():
            setattr(self, arg, val)
        if self.base_clf is None:
            self.base_clf = make_pipeline(LogisticRegression, **dict(class_weight="balanced"))
        
        if (self.meta_clf is None) and ensemble_type is 'stacking':
            self.meta_clf = LogisticRegression(class_weight="balanced")
                
        if confidence_threshold is not None:
            self.confidence_threshold = confidence_threshold
    
    def fit(self, X_train, y_train, sensors=None):
        if sensors:
            self.sensors = sensors
        
        if self.ensemble_type is 'bagging':
            self.bag_fit(X_train, y_train)
            
        if self.ensemble_type is 'stacking':
            self.stack_fit(X_train, y_train)
    
    def stack_fit(self, X_train, y_train):
        # split training data
        X_train_base, X_train_meta, y_train_base, y_train_meta = train_test_split(X_train, y_train, test_size=0.5)
        
        # train base classifiers
        self.bag_fit(X_train, y_train)
        y_pred_base = self.get_base_predictions(X_train_meta)

        # train meta classifier
        self.meta_clf.fit(y_pred_base, y_train_meta)

    def  bag_fit(self, X_train, y_train):
        for sensor in self.sensors:
            if self.verbose:
                print("Training classifier with data from %s sensor" % sensor)
            X_train_sensor = self.get_features_for_sensor(X_train, sensor)
            clf_sensor = clone(self.base_clf)
            if self.base_params is not None:
                clf_sensor.set_params(**self.base_params)
                
            clf_sensor.fit(X_train_sensor, y_train)
            self.classifiers[sensor] = clf_sensor
    
    def get_features_for_sensor(self,X, sensor):
        feature_cols = []
        
        for col in X.columns:
            sensor_names = es.sensor_key_dict[sensor]
            for sensor_name in sensor_names:
                if sensor_name in col:
                    feature_cols.append(col)
                    break
        
        return X[X.columns.intersection(feature_cols)]
    
    def predict(self, X_test, sensors=None):
        if sensors is None:
            sensors = self.sensors
        
        if self.ensemble_type is 'bagging':
            return self.bag_predict(X_test)
        
        if self.ensemble_type is 'stacking':
            return self.stack_predict(X_test)
            
    def bag_predict(self, X_test):
        y_pred = get_base_predictions(X_test)
        # average across rows
        y_mean_pred = y_pred.mean(axis=1)
        y_pred = y_mean_pred > self.confidence_threshold # may have to convert this from boolean to integer 1,0
        return y_pred.astype(int)
    
    def stack_predict(self, X_test):
        y_base_pred_probas = self.get_base_predictions(X_test)
        y_meta_pred = self.meta_clf.predict(y_base_pred_probas)
        return y_meta_pred
    
    def get_base_predictions(self, X_test):
        predictions_by_classifier = {}
        
        for sensor in self.sensors:
            X_test_sensor = self.get_features_for_sensor(X_test, sensor)
            predictions = self.classifiers[sensor].predict_proba(X_test_sensor)[:,1]
            predictions_by_classifier[sensor] = pd.Series(predictions, name=sensor)

        predictions_df = pd.concat([p for p in predictions_by_classifier.values()], axis=1)
        
        return predictions_df

In [64]:
sensors = ["Acc", "Gyro", "Magnet", "WAcc", "Compass", "Loc", "Aud", "AP", "PS", "LF"]
clf = SensorFusionClassifier(sensors=sensors, ensemble_type='stacking')

In [65]:
context = "SITTING"
train_ind, test_ind = get_train_test_ind(folds[0], user_ids)

print("Training model...")
X_train = features_df.iloc[train_ind]
y_train = labels.iloc[train_ind]
y_train = np.array([1 if y == context else 0 for y in y_train])

X_test = features_df.iloc[test_ind]
y_test = labels.iloc[test_ind]
y_test = np.array([1 if y == context else 0 for y in y_test])

Training model...


In [66]:
clf.fit(X_train, y_train)

Training classifier with data from Acc sensor
Training classifier with data from Gyro sensor
Training classifier with data from Magnet sensor
Training classifier with data from WAcc sensor
Training classifier with data from Compass sensor
Training classifier with data from Loc sensor
Training classifier with data from Aud sensor
Training classifier with data from AP sensor
Training classifier with data from PS sensor
Training classifier with data from LF sensor


## ^^^^ Need to figure out how sklearn manages *args, **kwargs, **params, that kind of thing

# Use Late Fusion Averaging (Bagging)

In [43]:
test_metrics = test_model(SensorFusionClassifier, \
                          'SITTING', \
                          sensors=sensors)

Fold #0
Training model...
Training classifier with data from Acc sensor
Training classifier with data from Gyro sensor
Training classifier with data from Magnet sensor
Training classifier with data from WAcc sensor
Training classifier with data from Compass sensor
Training classifier with data from Loc sensor
Training classifier with data from Aud sensor
Training classifier with data from AP sensor
Training classifier with data from PS sensor
Training classifier with data from LF sensor
Testing model...
----------
Accuracy*:         0.76
Sensitivity (TPR): 0.81
Specificity (TNR): 0.72
Balanced accuracy: 0.77
Precision**:       0.72
----------
Fold #1
Training model...
Training classifier with data from Acc sensor
Training classifier with data from Gyro sensor
Training classifier with data from Magnet sensor
Training classifier with data from WAcc sensor
Training classifier with data from Compass sensor
Training classifier with data from Loc sensor
Training classifier with data from Aud

In [44]:
get_mean_metrics(test_metrics)

----------
Accuracy*:         0.74
Sensitivity (TPR): 0.77
Specificity (TNR): 0.71
Balanced accuracy: 0.74
Precision**:       0.69
----------


# Use Learned Weights (Stacking)

In [67]:
test_metrics = test_model(SensorFusionClassifier, \
                          'SITTING', \
                          sensors=sensors, ensemble_type="stacking")

Fold #0
Training model...
Training classifier with data from Acc sensor
Training classifier with data from Gyro sensor
Training classifier with data from Magnet sensor
Training classifier with data from WAcc sensor
Training classifier with data from Compass sensor
Training classifier with data from Loc sensor
Training classifier with data from Aud sensor
Training classifier with data from AP sensor
Training classifier with data from PS sensor
Training classifier with data from LF sensor
Testing model...
----------
Accuracy*:         0.77
Sensitivity (TPR): 0.82
Specificity (TNR): 0.72
Balanced accuracy: 0.77
Precision**:       0.72
----------
Fold #1
Training model...
Training classifier with data from Acc sensor
Training classifier with data from Gyro sensor
Training classifier with data from Magnet sensor
Training classifier with data from WAcc sensor
Training classifier with data from Compass sensor
Training classifier with data from Loc sensor
Training classifier with data from Aud

In [68]:
get_mean_metrics(test_metrics)

----------
Accuracy*:         0.74
Sensitivity (TPR): 0.78
Specificity (TNR): 0.71
Balanced accuracy: 0.74
Precision**:       0.69
----------
